<a href="https://colab.research.google.com/github/WeustiS/ProjectX/blob/main/Data_Parsing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests as r
from io import BytesIO, StringIO
import zipfile
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
import pandas as pd
import pickle
from google.colab import drive

In [ ]:
base_url = r'http://oasis.caiso.com/oasisapi/SingleZip?'
# ?queryname=PRC_AS&startdatetime=20191018T07:00-0000&enddatetime=20191019T07:00-0000
# &market_run_id=DAM&version=1&as_type=ALL&as_region=ALL

def get_data(params):
    # building URL
    url = base_url
    for key, value in params.items():
        url += f"{key}={value}&"
    data = r.get(url[:-1]).content
    print("URL:", url)
    try:
        filecontent = BytesIO(data)
    except TypeError:
        filecontent = StringIO(data)

    try:
            # have zipfile
        z = zipfile.ZipFile(filecontent)
    except zipfile.BadZipfile:
        return None

        # have unzipped content
    unzipped = [z.read(thisfile) for thisfile in z.namelist()]
    z.close()

        # return
    return unzipped

In [ ]:
# function to convert XML to 
def parse_XML(root):
  df = pd.DataFrame(columns=['Variable', 'Node', 'Date','Interval Number','Value'])
  j = 0
  #
  for i in root[1][0][4][1:]:
    #print(i[2].text, float(i[6].text))\
    new_row = {'Variable':i[0].text, 'Node':i[1].text, 'Date':i[2].text, 'Interval Number':float(i[3].text),'Value':float(i[6].text)}
    #append row to the dataframe
    df = df.append(new_row, ignore_index=True)
  return df

In [ ]:
payload = {
    'queryname': 'PRC_INTVL_LMP', 
    'startdatetime': '20190919T07:00-0000',
    'enddatetime': '20190920T07:00-0000',
    'version': '1',
    'market_run_id': 'RTM',
    'node': 'LAPLMG1_7_B2'
}

In [ ]:
# trial code for 20180103T07:00-0000 as start date, the dates don't match, idk why
start_date = '2018010' + str(3)+'T07:00-0000'
end_date = '2018010' + str(5)+'T07:00-0000'
payload = {
      'queryname': 'PRC_INTVL_LMP', 
      'startdatetime': start_date,
      'enddatetime': end_date,
      'version': '1',
      'market_run_id': 'RTM',
      'node': 'LAPLMG1_7_B2'
}
xml = get_data(payload)
xml_to_string = str(xml[0],'utf-8')
root = ET.fromstring(xml_to_string)
print(root[1][0][4][1].text)
data = parse_XML(root)
data

URL: http://oasis.caiso.com/oasisapi/SingleZip?queryname=PRC_INTVL_LMP&startdatetime=20180103T07:00-0000&enddatetime=20180105T07:00-0000&version=1&market_run_id=RTM&node=LAPLMG1_7_B2&




,Variable,Node,Date,Interval Number,Value
0,LMP_PRC,LAPLMG1_7_B2,2018-01-02,277.0,38.40892
1,LMP_PRC,LAPLMG1_7_B2,2018-01-02,278.0,40.16764
2,LMP_PRC,LAPLMG1_7_B2,2018-01-02,279.0,38.79589
3,LMP_PRC,LAPLMG1_7_B2,2018-01-02,280.0,38.05715
4,LMP_PRC,LAPLMG1_7_B2,2018-01-02,281.0,35.97792
5,LMP_PRC,LAPLMG1_7_B2,2018-01-02,282.0,35.02972
6,LMP_PRC,LAPLMG1_7_B2,2018-01-02,283.0,34.49555
7,LMP_PRC,LAPLMG1_7_B2,2018-01-02,284.0,35.46150
8,LMP_PRC,LAPLMG1_7_B2,2018-01-02,285.0,37.01112
9,LMP_PRC,LAPLMG1_7_B2,2018-01-02,286.0,35.96660


In [ ]:
start_date = ""
end_date = ""
DF_list= list()
for i in range(1,30):
  if (i < 10):
    print(i)
    # changing the start and end date for scraping purposes
    start_date = '2018010' + str(i)+'T07:00-0000'
    end_date = '2018010' + str(i+1)+'T07:00-0000'
    print(start_date)
    print(end_date)
    # dynamically changing payload
    payload = {
      'queryname': 'PRC_INTVL_LMP', 
      'startdatetime': start_date,
      'enddatetime': end_date,
      'version': '1',
      'market_run_id': 'RTM',
      'node': 'LAPLMG1_7_B2'
    }
    xml = get_data(payload)
    # the next two lines are for debugging purposes
    display(type(xml))
    if (xml is None):
      continue
    # converting xml to dataframe
    xml_to_string = str(xml[0],'utf-8')
    root = ET.fromstring(xml_to_string)
    print(root[1][0][4][1].text)
    data = parse_XML(root)
    display(data.head(5))
    DF_list.append(data)
  else:
    start_date = '201801' + str(i)+'T07:00-0000'
    end_date = '201801' + str(i+1)+'T07:00-0000'
    print(start_date)
    print(end_date)
    payload = {
      'queryname': 'PRC_INTVL_LMP', 
      'startdatetime': start_date,
      'enddatetime': end_date,
      'version': '1',
      'market_run_id': 'RTM',
      'node': 'LAPLMG1_7_B2'
    }
    xml = get_data(payload)
    display(type(xml))
    if (xml is None):
      continue
    xml_to_string = str(xml[0],'utf-8')
    root = ET.fromstring(xml_to_string)
    data = parse_XML(root)
    display(data.head(5))
    DF_list.append(data)





1
20180101T07:00-0000
20180102T07:00-0000
URL: http://oasis.caiso.com/oasisapi/SingleZip?queryname=PRC_INTVL_LMP&startdatetime=20180101T07:00-0000&enddatetime=20180102T07:00-0000&version=1&market_run_id=RTM&node=LAPLMG1_7_B2&


list

,Variable,Node,Date,Interval Number,Value
0,LMP_PRC,LAPLMG1_7_B2,2017-12-31,277.0,48.65308
1,LMP_PRC,LAPLMG1_7_B2,2017-12-31,278.0,52.64113
2,LMP_PRC,LAPLMG1_7_B2,2017-12-31,279.0,48.65308
3,LMP_PRC,LAPLMG1_7_B2,2017-12-31,280.0,48.73813
4,LMP_PRC,LAPLMG1_7_B2,2017-12-31,281.0,48.73813


2
20180102T07:00-0000
20180103T07:00-0000
URL: http://oasis.caiso.com/oasisapi/SingleZip?queryname=PRC_INTVL_LMP&startdatetime=20180102T07:00-0000&enddatetime=20180103T07:00-0000&version=1&market_run_id=RTM&node=LAPLMG1_7_B2&


NoneType

...

NoneType

20180128T07:00-0000
20180129T07:00-0000
URL: http://oasis.caiso.com/oasisapi/SingleZip?queryname=PRC_INTVL_LMP&startdatetime=20180128T07:00-0000&enddatetime=20180129T07:00-0000&version=1&market_run_id=RTM&node=LAPLMG1_7_B2&


NoneType

20180129T07:00-0000
20180130T07:00-0000
URL: http://oasis.caiso.com/oasisapi/SingleZip?queryname=PRC_INTVL_LMP&startdatetime=20180129T07:00-0000&enddatetime=20180130T07:00-0000&version=1&market_run_id=RTM&node=LAPLMG1_7_B2&


NoneType

In [ ]:
# code to save data in google drive directly
data = parse_XML(root)
drive.mount('drive', force_remount=True)
data.to_csv('20190919.csv', index = False)
!cp 20190919.csv "drive/My Drive/ProjectX Data"

Mounted at drive
